# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
from holoviews import opts
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount gambier,-37.8333,140.7667,7.87,75,92,2.70,AU,1730393986
1,1,bathsheba,13.2167,-59.5167,31.34,66,40,5.66,BB,1730393987
2,2,port-aux-francais,-49.3500,70.2167,4.60,95,100,16.53,TF,1730393989
3,3,marsh harbour,26.5412,-77.0636,26.13,59,66,9.52,BS,1730393990
4,4,melfort,52.8667,-104.6177,-2.93,93,22,3.60,CA,1730393992


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    title="City_Data"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# Xpert Assistant helped me to modify my logical operators to return values for two conditions
ideal_vacay_df = (city_data_df[(
    city_data_df["Max Temp"] >= 22) & (city_data_df["Max Temp"] <= 30)
    & (city_data_df["Wind Speed"] >= 4) & (city_data_df["Wind Speed"] <= 24)
    & (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <=60)]
    )
 
# Drop any rows with null values
ideal_vacay_df = ideal_vacay_df.dropna(how="any")

# Display sample data
ideal_vacay_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,marsh harbour,26.5412,-77.0636,26.13,59,66,9.52,BS,1730393990
22,22,nouadhibou,20.9310,-17.0347,26.99,47,100,4.63,MR,1730394020
112,112,broome,42.2506,-75.8330,24.70,54,0,8.23,US,1730394174
199,199,jamestown,42.0970,-79.2353,23.34,60,75,7.20,US,1730394156
207,207,murzuq,25.9155,13.9184,23.70,30,7,4.88,LY,1730394339
275,275,talara,-4.5772,-81.2719,24.42,60,0,6.69,PE,1730394455
351,351,vredendal,-31.6683,18.5012,26.82,30,92,8.09,ZA,1730394577
365,365,batesburg-leesville,33.9101,-81.5373,26.87,59,8,4.45,US,1730394608
386,386,vice,-5.4167,-80.7833,26.39,49,27,5.33,PE,1730394643
413,413,san lorenzo,-25.3333,-57.5333,29.52,51,100,4.12,PY,1730394413


In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = ideal_vacay_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    title="Ideal Weather Locations"
)
    
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacay_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,marsh harbour,BS,26.5412,-77.0636,59,
22,nouadhibou,MR,20.9310,-17.0347,47,
112,broome,US,42.2506,-75.8330,54,
199,jamestown,US,42.0970,-79.2353,60,
207,murzuq,LY,25.9155,13.9184,30,
275,talara,PE,-4.5772,-81.2719,60,
351,vredendal,ZA,-31.6683,18.5012,30,
365,batesburg-leesville,US,33.9101,-81.5373,59,
386,vice,PE,-5.4167,-80.7833,49,
413,san lorenzo,PY,-25.3333,-57.5333,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
# my original code
radius = 10000 
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 30,
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
marsh harbour - nearest hotel: Lofty Fig Villas
nouadhibou - nearest hotel: Hotel valencia
broome - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
murzuq - nearest hotel: فندق باريس - مرزق
talara - nearest hotel: Hospedaje El Aypate
vredendal - nearest hotel: No hotel found
batesburg-leesville - nearest hotel: Leesville Lodge
vice - nearest hotel: No hotel found
san lorenzo - nearest hotel: Hotel del Sur
conde - nearest hotel: Paladino


,City,Country,Lat,Lng,Humidity,Hotel Name
3,marsh harbour,BS,26.5412,-77.0636,59,Lofty Fig Villas
22,nouadhibou,MR,20.9310,-17.0347,47,Hotel valencia
112,broome,US,42.2506,-75.8330,54,No hotel found
199,jamestown,US,42.0970,-79.2353,60,DoubleTree Jamestown
207,murzuq,LY,25.9155,13.9184,30,فندق باريس - مرزق
275,talara,PE,-4.5772,-81.2719,60,Hospedaje El Aypate
351,vredendal,ZA,-31.6683,18.5012,30,No hotel found
365,batesburg-leesville,US,33.9101,-81.5373,59,Leesville Lodge
386,vice,PE,-5.4167,-80.7833,49,No hotel found
413,san lorenzo,PY,-25.3333,-57.5333,51,Hotel del Sur


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
# Xpert Assistant showed me hover_cols=['Hotel Name']

map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols=['Hotel Name', 'Country'],
    title="Ideal Weather Locations"
)

# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)